# End to End Industrial IoT (IIoT) on Azure Databricks
## Part 1: Data Engineering
This notebook demonstrates the following architecture for IIoT Ingest, Processing and Analytics on Azure. The following architecture is implemented for the demo. 
<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/end_to_end_architecture.png" width=800>

The notebook is broken into sections following these steps:
1. **Data Ingest** - stream real-time raw sensor data from Azure IoT Hubs into the Delta format in Azure Storage
2. **Data Processing** - stream process sensor data from raw (Bronze) to silver (aggregated) to gold (enriched) Delta tables on Azure Storage

## Step 1 - Environment Setup

The pre-requisites are listed below:

Reference posts where I describe details of set for each

Note that secrets were removed from notebook and will be restored in a future update

Update on Maven coordinates for Azure Event Hubs Connector


### Azure Services Required
* Azure IoT Hub 
* [Azure IoT Simulator](https://azure-samples.github.io/raspberry-pi-web-simulator/) running with the code provided in [this github repo](https://github.com/tomatoTomahto/azure_databricks_iot/blob/master/azure_iot_simulator.js) and configured for your IoT Hub
* ADLS Gen 2 Storage account with a container called `iot`
* Azure Synapse SQL Pool call `iot`

### Azure Databricks Configuration Required -> THIS NEEDS REVISION
* 3-node (min) Databricks Cluster running **DBR 7.0+** and the following libraries:
 * **Azure Event Hubs Connector for Databricks** - Maven coordinates `com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.17`
* The following Secrets defined in scope `iot`
 * `iothub-cs` - Connection string for your IoT Hub **(Important - use the [Event Hub Compatible](https://devblogs.microsoft.com/iotdev/understand-different-connection-strings-in-azure-iot-hub/) connection string)**
 * `adls_key` - Access Key to ADLS storage account **(Important - use the [Access Key](https://raw.githubusercontent.com/tomatoTomahto/azure_databricks_iot/master/bricks.com/blog/2020/03/27/data-exfiltration-protection-with-azure-databricks.html))**
 * `synapse_cs` - JDBC connect string to your Synapse SQL Pool **(Important - use the [SQL Authentication](https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/synapse-analytics#spark-driver-to-azure-synapse) with username/password connection string)**
* The following notebook widgets populated:
 * `Storage Account` - Name of your storage account

In [0]:
#Initialize widgets
# AzureML Workspace info (name, region, resource group and subscription ID) for model deployment
# This doesn't have to be rerun after running once in the notebook. A second run, won't erase any content added into the text boxes
dbutils.widgets.text("Storage Account","","Storage Account")
dbutils.widgets.text("Event Hub Name","","Event Hub Name")
dbutils.widgets.text("ADLS Key", "", "ADLS Key")
dbutils.widgets.text("IOT CS", "", "IOT CS")
dbutils.widgets.text("Synapse JDBC", "", "Synapse JDBC")


In [0]:
# Setup access to storage account for temp data when pushing to Synapse
storage_account = dbutils.widgets.get("Storage Account")

# In the original notebook adls_key was stored as a secret. A future update will restore this approach
adls_key = dbutils.widgets.get("ADLS Key")

BLOB_CONTAINER_NAME = "iot"
spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", adls_key)

In [0]:
# Setup storage locations for all data
ROOT_PATH = f"abfss://iot@{storage_account}.dfs.core.windows.net/"
BRONZE_PATH = ROOT_PATH + "bronze/"
SILVER_PATH = ROOT_PATH + "silver/"
GOLD_PATH = ROOT_PATH + "gold/"
SYNAPSE_PATH = ROOT_PATH + "synapse/"
CHECKPOINT_PATH = ROOT_PATH + "checkpoints/"

print(BRONZE_PATH)

abfss://iot@afmstorewind001.dfs.core.windows.net/bronze/

In [0]:
# Other initializations and imports
# Connection String to Hub using Event Compatible name
IOT_CS = dbutils.widgets.get("IOT CS")

EVENTHUB_NAME = ""

#Encrypt the connection string
csEncrypt = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(IOT_CS)

#The Event Hub configuration
ehConf = { 
  'eventhubs.connectionString':csEncrypt,
  'ehName':dbutils.widgets.get("Event Hub Name")
}

# Enable auto compaction and optimized writes in Delta
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled","true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled","true")

# Pyspark and ML Imports
import os, json, requests
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [0]:
%sql
-- Clean up tables & views
-- Do not run this if you are reconnecting notebook and partially executing it
-- ONLY RUN IF STARTING OVER !!! 
DROP TABLE IF EXISTS turbine_raw;
DROP TABLE IF EXISTS weather_raw;
DROP TABLE IF EXISTS turbine_agg;
DROP TABLE IF EXISTS weather_agg;
DROP TABLE IF EXISTS turbine_enriched;
DROP TABLE IF EXISTS turbine_power;
DROP TABLE IF EXISTS turbine_maintenance;
DROP VIEW IF EXISTS turbine_combined;
DROP VIEW IF EXISTS feature_view;
DROP TABLE IF EXISTS turbine_life_predictions;
DROP TABLE IF EXISTS turbine_power_predictions;

In [0]:
# empty the iot container
# do not run this if you are re-entering the notebook and want to keep the current data
dbutils.fs.rm(ROOT_PATH, True)

Out[4]: True

In [0]:
# Make sure root path is empty - empties the iot container
# check the current status if empty it will output [] otherwise it will list the first level folders in iot container
dbutils.fs.ls(ROOT_PATH)
# use azure storage explorer to see what data exists



Out[5]: []

## Step 2 - Data Ingest from IoT Hubs
Azure Databricks provides a native connector to IoT and Event Hubs. Below, we will use PySpark Structured Streaming to read from an IoT Hub stream of data and write the data in it's raw format directly into Delta. 

Make sure that your IoT Simulator is sending payloads to IoT Hub as shown below.

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/iot_simulator.gif" width=800>

We have two separate types of data payloads in our IoT Hub:
1. **Turbine Sensor readings** - this payload contains `date`,`timestamp`,`deviceid`,`rpm` and `angle` fields
2. **Weather Sensor readings** - this payload contains `date`,`timestamp`,`temperature`,`humidity`,`windspeed`, and `winddirection` fields

We split out the two payloads into separate streams and write them both into Delta locations on Azure Storage. We are able to query these two Bronze tables *immediately* as the data streams in.

In [0]:
# start wind turbine simulator before running this....
# Schema of incoming data from IoT hub
schema = "timestamp timestamp, deviceId string, temperature double, humidity double, windspeed double, winddirection string, rpm double, angle double"

# Read directly from IoT Hub using the EventHubs library for Databricks
iot_stream = (
  spark.readStream.format("eventhubs")                                               # Read from IoT Hubs directly
    .options(**ehConf)                                                               # Use the Event-Hub-enabled connect string
    .load()                                                                          # Load the data
    .withColumn('reading', F.from_json(F.col('body').cast('string'), schema))        # Extract the "body" payload from the messages
    .select('reading.*', F.to_date('reading.timestamp').alias('date'))               # Create a "date" field for partitioning
)

# Split our IoT Hub stream into separate streams and write them both into their own Delta locations
write_turbine_to_delta = (
  iot_stream.filter('temperature is null')                                           # Filter out turbine telemetry from other data streams
    .select('date','timestamp','deviceId','rpm','angle')                             # Extract the fields of interest
    .writeStream.format('delta')                                                     # Write our stream to the Delta format
    .partitionBy('date')                                                             # Partition our data by Date for performance
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_raw")                   # Checkpoint so we can restart streams gracefully
    .start(BRONZE_PATH + "turbine_raw")                                              # Stream the data into an ADLS Path
)

write_weather_to_delta = (
  iot_stream.filter(iot_stream.temperature.isNotNull())                              # Filter out weather telemetry only
    .select('date','deviceid','timestamp','temperature','humidity','windspeed','winddirection') 
    .writeStream.format('delta')                                                     # Write our stream to the Delta format
    .partitionBy('date')                                                             # Partition our data by Date for performance
    .option("checkpointLocation", CHECKPOINT_PATH + "weather_raw")                   # Checkpoint so we can restart streams gracefully
    .start(BRONZE_PATH + "weather_raw")                                              # Stream the data into an ADLS Path
)

# Create the external tables once data starts to stream in
while True:
  try:
    spark.sql(f'CREATE TABLE IF NOT EXISTS turbine_raw USING DELTA LOCATION "{BRONZE_PATH + "turbine_raw"}"')
    spark.sql(f'CREATE TABLE IF NOT EXISTS weather_raw USING DELTA LOCATION "{BRONZE_PATH + "weather_raw"}"')
    break
  except:
    pass

In [0]:
# this is not used
# check BRONZEPATH for raw data before running
# check in azure storage explorer and run SQL below

In [0]:
# this is not used
# start wind turbine simulator before running this....
# Schema of incoming data from IoT hub
schema = "timestamp timestamp, deviceId string, temperature double, humidity double, windspeed double, winddirection string, rpm double, angle double"

# Read directly from IoT Hub using the EventHubs library for Databricks
iot_stream = (
  spark.readStream.format("eventhubs")                                               # Read from IoT Hubs directly
    .options(**ehConf)                                                               # Use the Event-Hub-enabled connect string
    .load()                                                                          # Load the data
    .withColumn('reading', F.from_json(F.col('body').cast('string'), schema))        # Extract the "body" payload from the messages
    .select('reading.*', F.to_date('reading.timestamp').alias('date'))               # Create a "date" field for partitioning
)

# Split our IoT Hub stream into separate streams and write them both into their own Delta locations
write_turbine_to_delta = (
  iot_stream.filter('temperature is null')                                           # Filter out turbine telemetry from other data streams
    .select('date','timestamp','deviceId','rpm','angle')                             # Extract the fields of interest
    .writeStream.format('delta')                                                     # Write our stream to the Delta format
    .partitionBy('date')                                                             # Partition our data by Date for performance
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_raw")                   # Checkpoint so we can restart streams gracefully
    .start(BRONZE_PATH + "turbine_raw")                                              # Stream the data into an ADLS Path
)

In [0]:
%sql 
-- We can query the data directly from storage immediately as soon as it starts streams into Delta 
SELECT * FROM turbine_raw WHERE deviceid = 'WindTurbine-1'

date,timestamp,deviceId,rpm,angle
2022-09-28,2022-09-28T23:06:15.000+0000,WindTurbine-1,9.009463547622591,7.883280604169768
2022-09-28,2022-09-28T23:06:35.000+0000,WindTurbine-1,7.234462540432943,7.3301547228788255
2022-09-28,2022-09-28T23:06:55.000+0000,WindTurbine-1,7.490962706954639,5.5545923685853085
2022-09-28,2022-09-28T23:07:15.000+0000,WindTurbine-1,7.7189126557963625,7.754048573821818
2022-09-28,2022-09-28T23:07:35.000+0000,WindTurbine-1,7.6543244596148785,5.697533902163018
2022-09-28,2022-09-28T23:07:55.000+0000,WindTurbine-1,7.858566268355756,5.876245484811285
2022-09-28,2022-09-28T23:08:15.000+0000,WindTurbine-1,7.115888843938621,7.226402738446294
2022-09-28,2022-09-28T23:08:35.000+0000,WindTurbine-1,7.737948535255762,5.770704968348791
2022-09-28,2022-09-28T23:08:55.000+0000,WindTurbine-1,7.30902712132723,6.395398731161326
2022-09-28,2022-09-28T23:09:15.000+0000,WindTurbine-1,7.282391563704153,7.372092618241133


In [0]:
#VARIOUS TESTS to se what happened


In [0]:
# see if any tables were created
spark.catalog.listTables(dbName="default")

Out[7]: [Table(name='adult', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='cleaned_taxes', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='cleaned_taxes_sparkr', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='departuredelayswindow', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='diamonds', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='ex001_ft', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='ex001_md', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='ex001_rd', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='ex001_ta', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='feature_table', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='fireservicecalls', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='gold_readings', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='iris', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='loan_stats_delta', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='loanstats3a_csv', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='news_articles_csv', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orderstbl', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='test_body', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='test_body2', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='test_deviceid', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='turbine_age', database='default', description=None, tableType='VIEW', isTemporary=False),
 Table(name='turbine_raw', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='userstbl', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='weather_raw', database='default', description=None, tableType='EXTERNAL', isTemporary=False)]

## Step 2 - Data Processing in Delta
While our raw sensor data is being streamed into Bronze Delta tables on Azure Storage, we can create streaming pipelines on this data that flow it through Silver and Gold data sets.

We will use the following schema for Silver and Gold data sets:

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/iot_delta_bronze_to_gold.png" width=800>

### 2a. Delta Bronze (Raw) to Delta Silver (Aggregated)
The first step of our processing pipeline will clean and aggregate the measurements to 1 hour intervals. 

Since we are aggregating time-series values and there is a likelihood of late-arriving data and data changes, we will use the [**MERGE**](https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/merge-into?toc=https%3A%2F%2Fdocs.microsoft.com%2Fen-us%2Fazure%2Fazure-databricks%2Ftoc.json&bc=https%3A%2F%2Fdocs.microsoft.com%2Fen-us%2Fazure%2Fbread%2Ftoc.json) functionality of Delta to upsert records into target tables. 

MERGE allows us to upsert source records into a target storage location. This is useful when dealing with time-series data as:
1. Data often arrives late and requires aggregation states to be updated
2. Historical data needs to be backfilled while streaming data is feeding into the table

When streaming source data, `foreachBatch()` can be used to perform a merges on micro-batches of data.

In [0]:
# Create functions to merge turbine and weather data into their target Delta tables
def merge_delta(incremental, target): 
  incremental.dropDuplicates(['date','window','deviceid']).createOrReplaceTempView("incremental")
  
  try:
    # MERGE records into the target table using the specified join key
    incremental._jdf.sparkSession().sql(f"""
      MERGE INTO delta.`{target}` t
      USING incremental i
      ON i.date=t.date AND i.window = t.window AND i.deviceId = t.deviceid
      WHEN MATCHED THEN UPDATE SET *
      WHEN NOT MATCHED THEN INSERT *
    """)
  except:
    # If the †arget table does not exist, create one
    incremental.write.format("delta").partitionBy("date").save(target)
    
turbine_b_to_s = (
  spark.readStream.format('delta').table("turbine_raw")                        # Read data as a stream from our source Delta table
    .groupBy('deviceId','date',F.window('timestamp','5 minutes'))              # Aggregate readings to hourly intervals
    .agg(F.avg('rpm').alias('rpm'), F.avg("angle").alias("angle"))
    .writeStream                                                               # Write the resulting stream
    .foreachBatch(lambda i, b: merge_delta(i, SILVER_PATH + "turbine_agg"))    # Pass each micro-batch to a function
    .outputMode("update")                                                      # Merge works with update mode
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_agg")             # Checkpoint so we can restart streams gracefully
    .start()
)

weather_b_to_s = (
  spark.readStream.format('delta').table("weather_raw")                        # Read data as a stream from our source Delta table
    .groupBy('deviceid','date',F.window('timestamp','5 minutes'))              # Aggregate readings to hourly intervals
    .agg({"temperature":"avg","humidity":"avg","windspeed":"avg","winddirection":"last"})
    .selectExpr('date','window','deviceid','`avg(temperature)` as temperature','`avg(humidity)` as humidity',
                '`avg(windspeed)` as windspeed','`last(winddirection)` as winddirection')
    .writeStream                                                               # Write the resulting stream
    .foreachBatch(lambda i, b: merge_delta(i, SILVER_PATH + "weather_agg"))    # Pass each micro-batch to a function
    .outputMode("update")                                                      # Merge works with update mode
    .option("checkpointLocation", CHECKPOINT_PATH + "weather_agg")             # Checkpoint so we can restart streams gracefully
    .start()
)

# Create the external tables once data starts to stream in
while True:
  try:
    spark.sql(f'CREATE TABLE IF NOT EXISTS turbine_agg USING DELTA LOCATION "{SILVER_PATH + "turbine_agg"}"')
    spark.sql(f'CREATE TABLE IF NOT EXISTS weather_agg USING DELTA LOCATION "{SILVER_PATH + "weather_agg"}"')
    break
  except:
    pass

In [0]:
# another merge_delta definition if not running previous streams
# not used
# Create functions to merge turbine and weather data into their target Delta tables
def merge_delta(incremental, target): 
  incremental.dropDuplicates(['date','window','deviceid']).createOrReplaceTempView("incremental")
  
  try:
    # MERGE records into the target table using the specified join key
    incremental._jdf.sparkSession().sql(f"""
      MERGE INTO delta.`{target}` t
      USING incremental i
      ON i.date=t.date AND i.window = t.window AND i.deviceId = t.deviceid
      WHEN MATCHED THEN UPDATE SET *
      WHEN NOT MATCHED THEN INSERT *
    """)
  except:
    # If the †arget table does not exist, create one
    incremental.write.format("delta").partitionBy("date").save(target)

In [0]:
# look for agg data in SILVER_PATH and run SQL below against the table

In [0]:
%sql
-- As data gets merged in real-time to our hourly table, we can query it immediately
SELECT * FROM turbine_agg t JOIN weather_agg w ON (t.date=w.date AND t.window=w.window) WHERE t.deviceid='WindTurbine-1' ORDER BY t.window DESC

deviceId,date,window,rpm,angle,date,window,deviceid,temperature,humidity,windspeed,winddirection
WindTurbine-1,2022-09-28,"List(2022-09-28T23:30:00.000+0000, 2022-09-28T23:35:00.000+0000)",8.19815517169443,7.030528632375485,2022-09-28,"List(2022-09-28T23:30:00.000+0000, 2022-09-28T23:35:00.000+0000)",WeatherCapture,26.15996350824847,69.06970677318483,6.9069706773184825,SW
WindTurbine-1,2022-09-28,"List(2022-09-28T23:25:00.000+0000, 2022-09-28T23:30:00.000+0000)",7.970652653833839,7.440987738771276,2022-09-28,"List(2022-09-28T23:25:00.000+0000, 2022-09-28T23:30:00.000+0000)",WeatherCapture,26.32083602489515,68.63000949195833,6.86300094919583,W
WindTurbine-1,2022-09-28,"List(2022-09-28T23:20:00.000+0000, 2022-09-28T23:25:00.000+0000)",7.959177161921575,6.697613350014712,2022-09-28,"List(2022-09-28T23:20:00.000+0000, 2022-09-28T23:25:00.000+0000)",WeatherCapture,25.53053844401338,70.52458685416791,7.052458685416797,W
WindTurbine-1,2022-09-28,"List(2022-09-28T23:15:00.000+0000, 2022-09-28T23:20:00.000+0000)",8.06617626663834,6.991237566641881,2022-09-28,"List(2022-09-28T23:15:00.000+0000, 2022-09-28T23:20:00.000+0000)",WeatherCapture,26.137963119911056,69.84821138331658,6.984821138331661,NW
WindTurbine-1,2022-09-28,"List(2022-09-28T23:10:00.000+0000, 2022-09-28T23:15:00.000+0000)",8.235157425216862,7.205762747064752,2022-09-28,"List(2022-09-28T23:10:00.000+0000, 2022-09-28T23:15:00.000+0000)",WeatherCapture,25.622499455767283,68.75427562958134,6.875427562958134,W
WindTurbine-1,2022-09-28,"List(2022-09-28T23:05:00.000+0000, 2022-09-28T23:10:00.000+0000)",7.651385104498448,6.694961966436142,2022-09-28,"List(2022-09-28T23:05:00.000+0000, 2022-09-28T23:10:00.000+0000)",WeatherCapture,26.094802304147,69.59576425995115,6.959576425995115,NW


### 2b. Delta Silver (Aggregated) to Delta Gold (Enriched)
Next we perform a streaming join of weather and turbine readings to create one enriched dataset we can use for data science and model training.

In [0]:
# Read streams from Delta Silver tables and join them together on common columns (date & window)
turbine_agg = spark.readStream.format('delta').option("ignoreChanges", True).table('turbine_agg')
weather_agg = spark.readStream.format('delta').option("ignoreChanges", True).table('weather_agg').drop('deviceid')
turbine_enriched = turbine_agg.join(weather_agg, ['date','window'])

# Write the stream to a foreachBatch function which performs the MERGE as before
merge_gold_stream = (
  turbine_enriched
    .selectExpr('date','deviceid','window.start as window','rpm','angle','temperature','humidity','windspeed','winddirection')
    .writeStream 
    .foreachBatch(lambda i, b: merge_delta(i, GOLD_PATH + "turbine_enriched"))
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_enriched")         
    .start()
)

# Create the external tables once data starts to stream in
while True:
  try:
    spark.sql(f'CREATE TABLE IF NOT EXISTS turbine_enriched USING DELTA LOCATION "{GOLD_PATH + "turbine_enriched"}"')
    break
  except:
    pass

In [0]:
%sql SELECT * FROM turbine_enriched WHERE deviceid='WindTurbine-1'

date,deviceid,window,rpm,angle,temperature,humidity,windspeed,winddirection
2022-09-28,WindTurbine-1,2022-09-28T23:10:00.000+0000,8.235157425216862,7.205762747064752,25.622499455767283,68.75427562958134,6.875427562958134,W
2022-09-28,WindTurbine-1,2022-09-28T23:20:00.000+0000,7.959177161921575,6.697613350014712,25.53053844401338,70.52458685416791,7.052458685416797,W
2022-09-28,WindTurbine-1,2022-09-28T23:30:00.000+0000,8.19815517169443,7.030528632375485,26.15996350824847,69.06970677318483,6.9069706773184825,SW
2022-09-28,WindTurbine-1,2022-09-28T23:15:00.000+0000,8.06617626663834,6.991237566641881,26.137963119911056,69.84821138331658,6.984821138331661,NW
2022-09-28,WindTurbine-1,2022-09-28T23:25:00.000+0000,7.970652653833839,7.440987738771276,26.32083602489515,68.63000949195833,6.86300094919583,W
2022-09-28,WindTurbine-1,2022-09-28T23:05:00.000+0000,7.651385104498448,6.694961966436142,26.094802304147,69.59576425995115,6.959576425995115,NW


### 2c: Stream Delta GOLD Table to Synapse
Synapse Analytics provides on-demand SQL directly on Data Lake source formats. Databricks can also directly stream data to Synapse SQL Pools for Data Warehousing workloads like BI dashboarding and reporting. 

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/synapse_databricks_delta.png" width=800>

In [0]:
#not needed ? 
#Configure CLI
#initially used this path
#/bin/bash: /databricks/conda/envs/databricks-ml/bin/databricks
#new path with MLOps-Cluster
! /databricks/python3/bin/databricks configure

Databricks Host (should begin with https://):

In [0]:
# from C:\devafm\afmbus\training\safari\LimitlessSynapse\20211123-Limitless-Analytics-with-Azure-Synapse.doc for XYZ

synapse_cs = dbutils.widgets.get("Synapse JDBC")


In [0]:
# originally had option("url",dbutils.secrets.get("iot","synapse_cs")) 
spark.conf.set("spark.databricks.sqldw.writeSemantics", "copy")                           # Use COPY INTO for faster loads to Synapse from Databricks
# have to manually start the iot pool ....

write_to_synapse = (
  spark.readStream.format('delta').option('ignoreChanges',True).table('turbine_enriched') # Read in Gold turbine readings from Delta as a stream
    .writeStream.format("com.databricks.spark.sqldw")                                     # Write to Synapse (SQL DW connector)
    .option("url",synapse_cs)                            # SQL Pool JDBC connection (with SQL Auth) string
    .option("tempDir", SYNAPSE_PATH)                                                      # Temporary ADLS path to stage the data (with forwarded permissions)
    .option("forwardSparkAzureStorageCredentials", "true")
    .option("dbTable", "turbine_enriched")                                                # Table in Synapse to write to
    .option("checkpointLocation", CHECKPOINT_PATH+"synapse")                              # Checkpoint for resilient streaming
    .start()
)

### 2d. Backfill Historical Data
In order to train a model, we will need to backfill our streaming data with historical data. The cell below generates 1 year of historical hourly turbine and weather data and inserts it into our Gold Delta table.

In [0]:
import pandas as pd
import numpy as np

# Function to simulate generating time-series data given a baseline, slope, and some seasonality
def generate_series(time_index, baseline, slope=0.01, period=365*24*12):
  rnd = np.random.RandomState(time_index)
  season_time = (time_index % period) / period
  seasonal_pattern = np.where(season_time < 0.4, np.cos(season_time * 2 * np.pi), 1 / np.exp(3 * season_time))
  return baseline * (1 + 0.1 * seasonal_pattern + 0.1 * rnd.randn(len(time_index)))
  
# Get start and end dates for our historical data
dates = spark.sql('select max(date)-interval 365 days as start, max(date) as end from turbine_enriched').toPandas()
  
# Get the baseline readings for each sensor for backfilling data
turbine_enriched_pd = spark.table('turbine_enriched').toPandas()
baselines = turbine_enriched_pd.min()[3:8]
devices = turbine_enriched_pd['deviceid'].unique()

# Iterate through each device to generate historical data for that device
print("---Generating Historical Enriched Turbine Readings---")
for deviceid in devices:
  print(f'Backfilling device {deviceid}')
  windows = pd.date_range(start=dates['start'][0], end=dates['end'][0], freq='5T') # Generate a list of hourly timestamps from start to end date
  historical_values = pd.DataFrame({
    'date': windows.date,
    'window': windows, 
    'winddirection': np.random.choice(['N','NW','W','SW','S','SE','E','NE'], size=len(windows)),
    'deviceId': deviceid
  })
  time_index = historical_values.index.to_numpy()                                 # Generate a time index

  for sensor in baselines.keys():
    historical_values[sensor] = generate_series(time_index, baselines[sensor])    # Generate time-series data from this sensor

  # Write dataframe to enriched_readings Delta table
  spark.createDataFrame(historical_values).write.format("delta").mode("append").saveAsTable("turbine_enriched")
  
# Create power readings based on weather and operating conditions
print("---Generating Historical Turbine Power Readings---")
spark.sql(f'CREATE TABLE turbine_power USING DELTA PARTITIONED BY (date) LOCATION "{GOLD_PATH + "turbine_power"}" AS SELECT date, window, deviceId, 0.1 * (temperature/humidity) * (3.1416 * 25) * windspeed * rpm AS power FROM turbine_enriched')

# Create a maintenance records based on peak power usage
print("---Generating Historical Turbine Maintenance Records---")
spark.sql(f'CREATE TABLE turbine_maintenance USING DELTA LOCATION "{GOLD_PATH + "turbine_maintenance"}" AS SELECT DISTINCT deviceid, FIRST(date) OVER (PARTITION BY deviceid, year(date), month(date) ORDER BY power) AS date, True AS maintenance FROM turbine_power')

---Generating Historical Enriched Turbine Readings---
Backfilling device WindTurbine-6
Backfilling device WindTurbine-9
Backfilling device WindTurbine-8
Backfilling device WindTurbine-4
Backfilling device WindTurbine-5
Backfilling device WindTurbine-1
Backfilling device WindTurbine-7
Backfilling device WindTurbine-3
Backfilling device WindTurbine-0
Backfilling device WindTurbine-2
---Generating Historical Turbine Power Readings---
---Generating Historical Turbine Maintenance Records---
Out[13]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# OPTIMIZE documentation -> https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-optimize.html
# have not yet run this as of 2022/05/04

In [0]:
%sql
-- Optimize all 3 tables for querying and model training performance
-- need a warning that this will take a long time .... Took 29.10 minutes when run on 05/05/22
OPTIMIZE turbine_enriched WHERE date<current_date() ZORDER BY deviceid, window;
OPTIMIZE turbine_power ZORDER BY deviceid, window;
OPTIMIZE turbine_maintenance ZORDER BY deviceid;

path,metrics
abfss://iot@afmstorewind001.dfs.core.windows.net/gold/turbine_maintenance,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1418), 0, List(0, 0), 0, null), 0, 1, 1, false)"


Our Delta Gold tables are now ready for predictive analytics! We now have hourly weather, turbine operating and power measurements, and daily maintenance logs going back one year. We can see that there is significant correlation between most of the variables.

In [0]:
%sql
-- Query all 3 tables
CREATE OR REPLACE VIEW gold_readings AS
SELECT r.*, 
  p.power, 
  ifnull(m.maintenance,False) as maintenance
FROM turbine_enriched r 
  JOIN turbine_power p ON (r.date=p.date AND r.window=p.window AND r.deviceid=p.deviceid)
  LEFT JOIN turbine_maintenance m ON (r.date=m.date AND r.deviceid=m.deviceid);
  
SELECT * FROM gold_readings ORDER BY deviceid, window

date,deviceid,window,rpm,angle,temperature,humidity,windspeed,winddirection,power,maintenance
2021-09-28,WindTurbine-0,2021-09-28T00:00:00.000+0000,8.082760247888954,7.024481258389998,27.265490780187687,73.29382790537916,7.329382790537912,NE,173.08678580857463,false
2021-09-28,WindTurbine-0,2021-09-28T00:05:00.000+0000,7.614942727533601,6.617915270621038,25.68740682163027,69.05169579005361,6.905169579005357,N,153.63062668867892,false
2021-09-28,WindTurbine-0,2021-09-28T00:10:00.000+0000,8.379000178481432,7.281934377944932,28.264793846043467,75.9801080653304,7.598010806533035,NE,186.0068417394524,false
2021-09-28,WindTurbine-0,2021-09-28T00:15:00.000+0000,8.074031293550496,7.016895189488368,27.23604548962645,73.2146744407884,7.321467444078837,N,172.71313849614816,false
2021-09-28,WindTurbine-0,2021-09-28T00:20:00.000+0000,8.142028399766238,7.075989407749016,27.465419418302485,73.83126679886831,7.383126679886828,W,175.63446624848436,false
2021-09-28,WindTurbine-0,2021-09-28T00:25:00.000+0000,8.217476763508863,7.141559287447561,27.719928596221262,74.51542656834047,7.451542656834044,N,178.90459241122502,false
2021-09-28,WindTurbine-0,2021-09-28T00:30:00.000+0000,8.270364953741772,7.187522806058454,27.8983357763121,74.99501247640575,7.499501247640572,SW,181.2148852680123,false
2021-09-28,WindTurbine-0,2021-09-28T00:35:00.000+0000,8.35898877229737,7.2645430730709935,28.19728957843233,75.79864622358116,7.579864622358112,N,185.11942952213482,false
2021-09-28,WindTurbine-0,2021-09-28T00:40:00.000+0000,8.820176813857216,7.665347582319781,29.753010385364224,79.98066275177582,7.998066275177578,E,206.11001841453967,false
2021-09-28,WindTurbine-0,2021-09-28T00:45:00.000+0000,9.105771620625003,7.91354934831399,30.716404366130924,82.57041377443856,8.257041377443851,SW,219.67368070959998,false


In [0]:
%sql
-- test if view always works
SELECT * FROM gold_readings ORDER BY deviceid, window

date,deviceid,window,rpm,angle,temperature,humidity,windspeed,winddirection,power,maintenance
2021-09-28,WindTurbine-0,2021-09-28T00:00:00.000+0000,8.082760247888954,7.024481258389998,27.265490780187687,73.29382790537916,7.329382790537912,NE,173.08678580857463,false
2021-09-28,WindTurbine-0,2021-09-28T00:05:00.000+0000,7.614942727533601,6.617915270621038,25.68740682163027,69.05169579005361,6.905169579005357,N,153.63062668867892,false
2021-09-28,WindTurbine-0,2021-09-28T00:10:00.000+0000,8.379000178481432,7.281934377944932,28.264793846043467,75.9801080653304,7.598010806533035,NE,186.0068417394524,false
2021-09-28,WindTurbine-0,2021-09-28T00:15:00.000+0000,8.074031293550496,7.016895189488368,27.23604548962645,73.2146744407884,7.321467444078837,N,172.71313849614816,false
2021-09-28,WindTurbine-0,2021-09-28T00:20:00.000+0000,8.142028399766238,7.075989407749016,27.465419418302485,73.83126679886831,7.383126679886828,W,175.63446624848436,false
2021-09-28,WindTurbine-0,2021-09-28T00:25:00.000+0000,8.217476763508863,7.141559287447561,27.719928596221262,74.51542656834047,7.451542656834044,N,178.90459241122502,false
2021-09-28,WindTurbine-0,2021-09-28T00:30:00.000+0000,8.270364953741772,7.187522806058454,27.8983357763121,74.99501247640575,7.499501247640572,SW,181.2148852680123,false
2021-09-28,WindTurbine-0,2021-09-28T00:35:00.000+0000,8.35898877229737,7.2645430730709935,28.19728957843233,75.79864622358116,7.579864622358112,N,185.11942952213482,false
2021-09-28,WindTurbine-0,2021-09-28T00:40:00.000+0000,8.820176813857216,7.665347582319781,29.753010385364224,79.98066275177582,7.998066275177578,E,206.11001841453967,false
2021-09-28,WindTurbine-0,2021-09-28T00:45:00.000+0000,9.105771620625003,7.91354934831399,30.716404366130924,82.57041377443856,8.257041377443851,SW,219.67368070959998,false


#### Benefits of Delta Lake on Time-Series Data
A key component of this architecture is the Azure Data Lake Store (ADLS), which enables the write-once, access-often analytics pattern in Azure. However, Data Lakes alone do not solve challenges that come with time-series streaming data. The Delta storage format provides a layer of resiliency and performance on all data sources stored in ADLS. Specifically for time-series data, Delta provides the following advantages over other storage formats on ADLS:

|**Required Capability**|**Other formats on ADLS**|**Delta Format on ADLS**|
|--------------------|-----------------------------|---------------------------|
|**Unified batch & streaming**|Data Lakes are often used in conjunction with a streaming store like CosmosDB, resulting in a complex architecture|ACID-compliant transactions enable data engineers to perform streaming ingest and historically batch loads into the same locations on ADLS|
|**Schema enforcement and evolution**|Data Lakes do not enforce schema, requiring all data to be pushed into a relational database for reliability|Schema is enforced by default. As new IoT devices are added to the data stream, schemas can be evolved safely so downstream applications don’t fail|
|**Efficient Upserts**|Data Lakes do not support in-line updates and merges, requiring deletion and insertions of entire partitions to perform updates|MERGE commands are effective for situations handling delayed IoT readings, modified dimension tables used for real-time enrichment, or if data needs to be reprocessed|
|**File Compaction**|Streaming time-series data into Data Lakes generate hundreds or even thousands of tiny files|Auto-compaction in Delta optimizes the file sizes to increase throughput and parallelism|
|**Multi-dimensional clustering**|Data Lakes provide push-down filtering on partitions only|ZORDERing time-series on fields like timestamp or sensor ID allows Databricks to filter and join on those columns up to 100x faster than simple partitioning techniques|